In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import os, glob

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/conorcorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

### First We'll find misses on the level of the organism instead of the encounter
This requires us to use code from the label generation notebook, but we won't collapse to the encounter level at the end.  

In [ ]:
# Define Fill in And Processing Functions
def fill_in_cefazolin(x):
    
    # If not missing just return what it is. 
    if x.Cefazolin == 'Susceptible' or x.Cefazolin == 'Resistant':
        return x.Cefazolin
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Cefazolin' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
    
    # If anything resistant to Cefepime or Ceftriaxone and Cefazolin not listed, assume resistant to Cefazolin
    if x.Cefepime == "Resistant" or x.Ceftriaxone == 'Resistant':
        return "Resistant"
    
    # For anything STREPTOCOCCUS related except STREPTOCOCCUS PNEUMONIAE  - refer to penicillin if not listed 
    if ('STREPTOCOCCUS' in x.organism or 'STREPTOCOCCI' in x.organism) and x.organism != 'STREPTOCOCCUS PNEUMONIAE':
        if x.Penicillin is not None:
            return x.Penicillin
    
    # Check For MRSA in STAPH AUREUS if not listed
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"

def fill_in_ceftriaxone(x):
    
    # If not missing just return what it is. 
    if x.Ceftriaxone == 'Susceptible' or x.Ceftriaxone == 'Resistant':
        return x.Ceftriaxone
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Ceftriaxone' and `Always Resistant` == 1")['bug'].values)
    
    # If anything resistant to Cefepime, assume resistant to Ceftriaxone
    if x.Cefepime == "Resistant":
        return "Resistant"
    
    # If susceptible to Cefazolin, assume susceptible to Ceftriaxone
    if x.Cefazolin == "Susceptible": 
        return "Susceptible"
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"

    # For anything STREPTOCOCCUS related - refer to penicillin if not listed
    if ('STREPTOCOCCUS' in x.organism or 'STREPTOCOCCI' in x.organism):
        if x.Penicillin is not None:
            return x.Penicillin
        
    # Check For MRSA in STAPH AUREUS 
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"

def fill_in_cefepime(x):
    
    # If not missing just return what it is. 
    if x.Cefepime == 'Susceptible' or x.Cefepime == 'Resistant':
        return x.Cefepime
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Cefepime' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
    
    # For anything STREPTOCOCCUS related - refer to penicillin 
    if ('STREPTOCOCCUS' in x.organism or 'STREPTOCOCCI' in x.organism):
        if x.Penicillin is not None:
            return x.Penicillin
        
    # Check For MRSA in STAPH AUREUS 
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
    
    # If susceptible to Cefazolin or Ceftriaxone assume susceptible to Cefepime
    if x.Cefazolin == "Susceptible" or x.Ceftriaxone == "Susceptible":
        return "Susceptible"
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"

def fill_in_zosyn(x):
     
    # If not missing just return what it is. 
    if x.Zosyn == 'Susceptible' or x.Zosyn == 'Resistant':
        return x.Zosyn
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules
                 .query("antibiotic == 'Piperacillin/Tazobactam' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
    
    # For anything STREPTOCOCCUS related - refer to penicillin 
    if ('STREPTOCOCCUS' in x.organism or 'STREPTOCOCCI' in x.organism):
        if x.Penicillin is not None:
            return x.Penicillin
        
    # Check For MRSA in STAPH AUREUS 
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
        
    # If susceptible to ampicillin, then zosyn susceptible - useless rule because of the following rule
    if x.Ampicillin == "Susceptible":
        return x.Ampicillin
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"


def fill_in_vancomycin(x):
    # If not missing just return what it is. 
    if x.Vancomycin == 'Susceptible' or x.Vancomycin == 'Resistant':
        return x.Vancomycin
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Vancomycin' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
    
    # Will apply Nancy Watz Rule Later (assume susc if not listed)
    return "Susceptible"

def fill_in_meropenem(x):
    # If not missing just return what it is. 
    if x.Meropenem == 'Susceptible' or x.Meropenem == 'Resistant':
        return x.Meropenem
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Meropenem' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
    
    # Check For MRSA in STAPH AUREUS 
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
        
    # If susceptible to ampicillin, then meropenem susceptible
    if x.Ampicillin == "Susceptible": # not the case with Enteroccocus, but this should be in alwasys resistant list above. 
        return x.Ampicillin # also - useless rule because of rule below
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"

def fill_in_ciprofloxacin(x):
    # If not missing just return what it is. 
    if x.Ciprofloxacin == 'Susceptible' or x.Ciprofloxacin == 'Resistant':
        return x.Ciprofloxacin
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Ciprofloxacin' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
    
    # If Resistant to Levofloxacin, resistant to Cipro
    if x.Levofloxacin == "Resistant":
        return "Resistant"
    
    # Check for MRSA - assume same as oxacillin if not listed
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"

def fill_in_ampicillin(x):
    
    # If not missing just return what it is. 
    if x.Ampicillin == 'Susceptible' or x.Ampicillin == 'Resistant':
        return x.Ampicillin
    
    # Read in bug rules
    df_bug_rules = pd.read_csv('resistant_bugs.csv')
    r_bugs = set(df_bug_rules.query("antibiotic == 'Ampicillin' and `Always Resistant` == 1")['bug'].values)
    
    # Uses hand labelled list of resistant bugs
    if x.organism in r_bugs:
        return "Resistant"
        
    # Check for MRSA - assume same as oxacillin if not listed
    if x.organism == 'STAPHYLOCOCCUS AUREUS':
        if x.Oxacillin is not None:
            return x.Oxacillin 
    
    # Otherwise apply Nancy Watz Rule (assume susc if not listed)
    return "Susceptible"


def combine_labels(arr):
    """
    Sometimes organisms have multiple suscept labels. When they do, if any of them are not one of
    Susceptible, Positive, or Susceptible - Dose Dependent then we say organism is resistant to 
    said antibiotic
    """
    for a in arr:
        if a not in ['Susceptible', 'Positive', 'Susceptible - Dose Dependent']:
            return 'Resistant'
    return 'Susceptible'

def combine_antibiotic_syns(x):
    """
    Antibiotics often are given different names in the AST tables, this function combines
    synnonyms so that we don't have to refer to multiple names in downstream analysis. 
    """
    if x == 'Aztreonam.':
        return 'Aztreonam'
    elif x == 'Cefazolin..':
        return 'Cefazolin'
    elif x == 'Ceftazidime.':
        return 'Ceftazidime'
    elif x in ('Ceftriaxone (Meningeal)', 'Ceftriaxone (Non-Meningeal)', 'Ceftriaxone.'):
        return 'Ceftriaxone'
    elif x in ('Ciprofloxacin.'):
        return 'Ciprofloxacin'
    elif x == 'Gentamicin 500 mcg/ml.':
        return 'Gentamicin'
    elif x in ('Oxacillin Screen', 'Oxacillin.'):
        return 'Oxacillin'
    elif x in ('PENICILLIN G (MENINGEAL)','PENICILLIN G (NON-MENINGEAL)', 'PENICILLIN V (ORAL)', 'Penicillin..'):
        return 'Penicillin'
    elif x == 'Trimethoprim/Sulfamethoxazole.':
        return 'Trimethoprim/Sulfamethoxazole'
    else:
        return x

In [ ]:
import pdb
# Query positive culture data
query = """
SELECT DISTINCT 
    cohort.pat_enc_csn_id_coded, cults.order_proc_id_coded, cults.description,
    cs.sens_organism_sid, cs.line, cs.organism,
    cs.antibiotic, cs.suscept, cs.sensitivity_value, cs.sens_ref_range 
FROM 
    `mining-clinical-decisions.abx.culture_orders_within_24_hrs` cults
INNER JOIN 
    `mining-clinical-decisions.abx.cohort_not_infected_rules` cohort
USING 
    (pat_enc_csn_id_coded)
INNER JOIN 
    `shc_core.culture_sensitivity` cs
USING 
    (order_proc_id_coded)
WHERE 
    organism not LIKE "%CANDIDA%"
AND 
    organism not in ('COAG NEGATIVE STAPHYLOCOCCUS', 'HAEMOPHILUS INFLUENZAE', 'HAEMOPHILUS PARAINFLUENZAE')
ORDER BY 
    cohort.pat_enc_csn_id_coded, cults.order_proc_id_coded, cs.line
"""

query_job = client.query(query)
df=query_job.to_dataframe()
df.head()

In [ ]:
filter_cols = ['example', 'pat_enc_csn_id_coded', 'organism', 'description', 'order_proc_id_coded', 'sens_organism_sid',
               'Cefepime', 'Ceftriaxone', 'Cefazolin', 'Vancomycin', 'Meropenem', 'Zosyn',
               'Ampicillin', 'Penicillin', 'Oxacillin', 'Ciprofloxacin', 'Levofloxacin']

df_wide = (df
    .assign(example=lambda x: x.pat_enc_csn_id_coded.map(str) + ', ' \
            + x.order_proc_id_coded.map(str) + ', ' + x.sens_organism_sid + ', ' + x.organism + ', ' + x.description,
            antibiotic=lambda x: [combine_antibiotic_syns(a) for a in x.antibiotic])
    .groupby(['example', 'antibiotic']) 
    .agg({'suscept' : lambda x: combine_labels(x)})          
    .reset_index()
    .pivot(index='example', columns='antibiotic', values='suscept')
    .reset_index()
    .assign(organism = lambda x: [a.split(', ')[3] for a in x.example],
            order_proc_id_coded = lambda x: [a.split(', ')[1] for a in x.example],
            sens_organism_sid = lambda x: [a.split(', ')[2] for a in x.example],
            pat_enc_csn_id_coded = lambda x: [a.split(', ')[0] for a in x.example],
            description = lambda x: [a.split(', ')[4] for a in x.example])
    .rename(columns={'Piperacillin/Tazobactam' : 'Zosyn'})
)
df_wide = df_wide[filter_cols]
df_wide.head()


In [ ]:
df_wide = (df_wide
    .assign(Ampicillin=lambda x: df_wide.apply(lambda x: fill_in_ampicillin(x), axis=1),
            Cefepime=lambda x: df_wide.apply(lambda x: fill_in_cefepime(x), axis=1),
            Ceftriaxone=lambda x: df_wide.apply(lambda x: fill_in_ceftriaxone(x), axis=1),
            Cefazolin=lambda x: df_wide.apply(lambda x: fill_in_cefazolin(x), axis=1),
            Ciprofloxacin=lambda x: df_wide.apply(lambda x: fill_in_ciprofloxacin(x), axis=1),
            Zosyn=lambda x: df_wide.apply(lambda x: fill_in_zosyn(x), axis=1),
            Vancomycin=lambda x: df_wide.apply(lambda x: fill_in_vancomycin(x), axis=1),
            Meropenem=lambda x: df_wide.apply(lambda x: fill_in_meropenem(x), axis=1),
            Vancomycin_Ceftriaxone=lambda x: ["Susceptible" if a == "Susceptible" 
                                              or b == "Susceptible" else "Resistant"
                                              for a, b in zip(x.Vancomycin, x.Ceftriaxone)],
            Vancomycin_Cefepime=lambda x: ["Susceptible" if a == "Susceptible" 
                                           or b == "Susceptible" else "Resistant"
                                           for a, b in zip(x.Vancomycin, x.Cefepime)],
            Vancomycin_Zosyn=lambda x: ["Susceptible" if a == "Susceptible" 
                                        or b == "Susceptible" else "Resistant"
                                        for a, b in zip(x.Vancomycin, x.Zosyn)],
            Vancomycin_Meropenem=lambda x: ["Susceptible" if a == "Susceptible" 
                                            or b == "Susceptible" else "Resistant"
                                            for a, b in zip(x.Vancomycin, x.Meropenem)])
    .reset_index()
)
df_wide.head()


In [ ]:
# Fill in any NA's with Susceptible - nancy watz rule
abx_map = {'Ceftriaxone' : "CEFTRIAXONE",
           'Vancomycin_Zosyn' : "PIPERACILLIN-TAZOBACTAM VANCOMYCIN",
           'Zosyn' : "PIPERACILLIN-TAZOBACTAM",
           'Vancomycin_Ceftriaxone' : "CEFTRIAXONE VANCOMYCIN",
           'Vancomycin_Cefepime' : "CEFEPIME VANCOMYCIN",
           'Cefepime' : "CEFEPIME",
           'Vancomycin' :  "VANCOMYCIN",
           'Meropenem' : "MEROPENEM",
           'Vancomycin_Meropenem' : "MEROPENEM VANCOMYCIN",
           'Cefazolin' : "CEFAZOLIN",
           'Ciprofloxacin' : "CIPROFLOXACIN",
           'Ampicillin' : 'AMPICILLIN'
          }
df_wide[list(abx_map.keys())] = df_wide[list(abx_map.keys())].fillna('Susceptible')
df_ast_labels_by_bug = df_wide[['pat_enc_csn_id_coded', 'organism', 'order_proc_id_coded', 'sens_organism_sid'] + list(abx_map.keys())]
df_ast_labels_by_bug.head()


In [ ]:
abx_map = {'Ceftriaxone' : "CEFTRIAXONE",
           'Vancomycin_Zosyn' : "PIPERACILLIN-TAZOBACTAM VANCOMYCIN",
           'Zosyn' : "PIPERACILLIN-TAZOBACTAM",
           'Vancomycin_Ceftriaxone' : "CEFTRIAXONE VANCOMYCIN",
           'Vancomycin_Cefepime' : "CEFEPIME VANCOMYCIN",
           'Cefepime' : "CEFEPIME",
           'Vancomycin' :  "VANCOMYCIN",
           'Meropenem' : "MEROPENEM",
           'Vancomycin_Meropenem' : "MEROPENEM VANCOMYCIN",
           'Cefazolin' : "CEFAZOLIN",
           'Ciprofloxacin' : "CIPROFLOXACIN",
           'Ampicillin' : 'AMPICILLIN'
          }
abx_map_inverse = {abx_map[key] : key for key in abx_map}
abx_map_inverse['CEFTRIAXONE PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Zosyn'
# abx_map_inverse['LEVOFLOXACIN PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Zosyn'
abx_map_inverse['AZITHROMYCIN PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Zosyn'
# abx_map_inverse['MEROPENEM PIPERACILLIN-TAZOBACTAM VANCOMYCIN'] = 'Vancomycin_Meropenem'
abx_map_inverse['AZITHROMYCIN CEFTRIAXONE'] = 'Ceftriaxone'

query = """
SELECT
    om.anon_id, om.pat_enc_csn_id_coded, om.order_med_id_coded, l.index_time, om.med_description,
    l.Ampicillin, l.Ciprofloxacin, l.Cefazolin, l.Ceftriaxone, l.Cefepime, l.Zosyn, l.Vancomycin,
    l.Meropenem, l.Vancomycin_Meropenem, l.Vancomycin_Zosyn, l.Vancomycin_Cefepime, l.Vancomycin_Ceftriaxone
FROM
    `mining-clinical-decisions.abx.abx_orders_given_and_stopped` om
INNER JOIN 
    `mining-clinical-decisions.abx.final_ast_labels` l
USING
    (pat_enc_csn_id_coded)
WHERE
    om.was_given = 1
ORDER BY 
    om.anon_id, om.pat_enc_csn_id_coded, om.order_time
"""
query_job = client.query(query)
df_abx = query_job.result().to_dataframe()
df_abx.head()

# Lambda that aggregates Antibiotic orders after we've grouped by CSN
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))

# Aggregate abx orders
df_drugs = (df_abx
    .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description]) # Only Take first word (abx)
    .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                        .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                    for a in x.med_description])
    .assign(year=lambda x: x.index_time.dt.year) # get year of each CSN - used to filter later on
    .groupby('pat_enc_csn_id_coded')
    .agg({'med_description' : concat_abx,
        'year' : 'first',
        'Ampicillin' : 'first',
        'Ciprofloxacin' : 'first',
        'Cefazolin' : 'first',
        'Ceftriaxone' : 'first',
        'Cefepime' : 'first',
        'Zosyn' : 'first',
        'Vancomycin' : 'first',
        'Meropenem' : 'first',
        'Vancomycin_Ceftriaxone' : 'first',
        'Vancomycin_Cefepime' : 'first',
        'Vancomycin_Zosyn' : 'first',
        'Vancomycin_Meropenem' : 'first'})
    .reset_index()
    # Only look at test set data and CSNs where allowed antibiotic selection was administered
    .query("year == 2019 and med_description in @abx_map_inverse", engine='python') 
    .assign(med_description=lambda x: [abx_map_inverse[a] for a in x.med_description])
)

In [ ]:
### Solve linear programming formulation so we can stratify by misses of optimized procedure as well
import sys
sys.path.insert(0, "../decision_alg")
from DecisionAnalysis import AbxDecisionMaker, load_predictions
from integer_programming import get_clinician_prescribing_patterns

abx_settings = {"Ceftriaxone" : 404,
    "Vancomycin_Zosyn" :  149,
    "Zosyn" : 102,
    "Vancomycin_Ceftriaxone" : 31,
    "Vancomycin_Cefepime" : 23,
    "Cefepime" : 14,
    "Vancomycin" : 13,
    "Vancomycin_Meropenem" : 9,
    "Meropenem" : 9,
    "Cefazolin" : 8,
    "Ciprofloxacin" : 8,
    "Ampicillin" : 0,
    }

df_predictions = load_predictions()
df_drugs = get_clinician_prescribing_patterns()
opt = AbxDecisionMaker(df_predictions, df_drugs, abx_settings)

In [ ]:
opt.solve_and_assign()

In [ ]:
# Map med_description to df_ast_labels_by_bug
df_ast_labels_by_bug = (df_ast_labels_by_bug
    .assign(pat_enc_csn_id_coded = lambda x: x.pat_enc_csn_id_coded.astype(int))
    .merge(opt.df[['pat_enc_csn_id_coded', 'med_description', 'IP_med_description']],
           how='inner',
           on='pat_enc_csn_id_coded')
)
df_ast_labels_by_bug.head()


In [ ]:
# Abx labels to binary
to_binary = lambda x: 1 if x == "Susceptible" else 0
for key in abx_map.keys():
    df_ast_labels_by_bug[key] = [1 if x == "Susceptible" else 0 for x in df_ast_labels_by_bug[key].values]
df_ast_labels_by_bug.head()

In [ ]:
def compute_was_covered(x, decision_column='med_description'):
    """
    Given med description, find appropriate label column and return whether patient was covered during CSN
    Returns "Not in abx options" if abx regimen isn't in our set of 12 options - useful for filtering later
    """
    if decision_column == 'med_description':
        med_description = x.med_description
    elif decision_column == 'random_med_description':
        med_description = x.random_med_description
    elif decision_column == 'IP_med_description':
        med_description = x.IP_med_description

    return x[med_description]

df_ast_labels_by_bug = (df_ast_labels_by_bug
    .assign(dr_covered=df_ast_labels_by_bug.apply(lambda x: compute_was_covered(x), axis=1))
    .assign(alg_covered=df_ast_labels_by_bug.apply(lambda x: compute_was_covered(x, 'IP_med_description'), axis=1))
)

In [ ]:
### Query culture sensitivity table for culture types
query = """
SELECT DISTINCT
    description, order_proc_id_coded 
FROM 
    `mining-clinical-decisions.abx.culture_orders_within_24_hrs` cult_orders
"""
query_job = client.query(query)
df_culture_types = query_job.result().to_dataframe()
df_culture_types['order_proc_id_coded'] = df_culture_types['order_proc_id_coded'].astype(int)

In [ ]:
df_ast_labels_by_bug.head()

In [ ]:
df_ast_labels_by_bug['order_proc_id_coded'] = df_ast_labels_by_bug['order_proc_id_coded'].astype(int)
df_ast_labels_by_bug = (df_ast_labels_by_bug
                        .merge(df_culture_types, on='order_proc_id_coded', how='left')
                       )
df_ast_labels_by_bug.head()

#### Get most common doctor misses

In [ ]:
import pdb
        
def pipe_func(df):
    df = df.assign(organism_and_culture = df.apply(lambda x: x['description'] + ' ' + x['organism'], axis=1))
    return df

def format_description(x):
    if "URINE" in x:
        return 'Urine Culture'
    elif "BLOOD" in x:
        return "Blood Culture"
    else:
        return "Other Fluid Culture"
df_frequent_misses = (df_ast_labels_by_bug
.query("dr_covered == 0", engine='python')
.assign(description = lambda x: [format_description(a) for a in x.description])
.assign(organism = lambda x: [a.capitalize() for a in x.organism])
.groupby(['description', 'organism'])
.agg(num_misses=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['description', 'num_misses'], ascending=False)
)
df_frequent_misses

#### Get most common alg misses

In [ ]:
df_frequent_misses_alg = (df_ast_labels_by_bug
.query("alg_covered == 0", engine='python')
.assign(description = lambda x: [format_description(a) for a in x.description])
.assign(organism = lambda x: [a.capitalize() for a in x.organism])
.groupby(['description', 'organism'])
.agg(num_misses=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['description', 'num_misses'], ascending=False)
)
df_frequent_misses_alg

In [ ]:
map_high_level_organism = {
    'Escherichia coli' : 'Lactose Fermenting GNRs',
    'Enterococcus species' : 'Enterococcus species',
    'Enterococcus faecalis' : 'Enterococcus species',
    'Enterobacter cloacae complex' : "Lactose Fermenting GNRs",
    'Klebsiella pneumoniae' : "Lactose Fermenting GNRs",
    'Morganella morganii' : "Non Lactose Fermenting GNRs",
    'Enterococcus faecium' : 'Enterococcus species',
    'Citrobacter freundii complex' : "Lactose Fermenting GNRs",
    'Mucoid pseudomonas aeruginosa' : 'Non Lactose Fermenting GNRs',
    'Proteus mirabilis' : 'Non Lactose Fermenting GNRs',
    'Proteus vulgaris group' : 'Non Lactose Fermenting GNRs',
    'Zzzcitrobacter amalonaticus' : 'Non Lactose Fermenting GNRs', 
    'Elizabethkingia species' : 'Non Lactose Fermenting GNRs',
    'Serratia marcescens' : 'Non Lactose Fermenting GNRs',
    'Stenotrophomonas maltophilia' : 'Non Lactose Fermenting GNRs',
    'Non fermenting gram negative rods' : 'Non Lactose Fermenting GNRs',
    'Streptococcus parasanguinis' : 'Streptococcus species',
    'Streptococcus mitis group' : 'Streptococcus species',
    'Viridans group streptococci not s. pneumoniae' : 'Streptococcus species',
    'Pseudomonas aeruginosa' : 'Non Lactose Fermenting GNRs',
    'Klebsiella aerogenes' : 'Lactose Fermenting GNRs',
    'Klebsiella oxytoca' : 'Lactose Fermenting GNRs',
    'Staph aureus {mrsa}' : 'MRSA',
    'Streptococcus species' : 'Streptococcus species',
    'Enterococcus gallinarum' : 'Enterococcus species',
    'Streptococcus sanguinis' : 'Streptococcus species',
    'Streptococcus salivarius group' : 'Streptococcus species',
    'Citrobacter species' : 'Lactose Fermenting GNRs',
}

### Group doctor misses into high level organisms

In [ ]:
df_frequent_misses_high_level = (df_ast_labels_by_bug
.query("dr_covered == 0", engine='python')
.assign(description = lambda x: [format_description(a) for a in x.description])
.assign(organism = lambda x: [a.capitalize() for a in x.organism])
.assign(organism=lambda x: [map_high_level_organism[a] for a in x.organism])
.groupby(['description', 'organism'])
.agg(num_misses=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['num_misses'], ascending=False)
.rename(columns={'description' : 'Culture Type',
                 'organism' : 'Infection Type',
                 'num_misses' : 'Number of Misses'})
)
df_frequent_misses_high_level.to_csv('doctor_misses_by_infection_type.csv', index=None)
df_frequent_misses_high_level

### Group Alg misses into high level organisms

In [ ]:
df_frequent_misses_alg_high_level = (df_ast_labels_by_bug
.query("alg_covered == 0", engine='python')
.assign(description = lambda x: [format_description(a) for a in x.description])
.assign(organism = lambda x: [a.capitalize() for a in x.organism])
.assign(organism=lambda x: [map_high_level_organism[a] for a in x.organism])
.groupby(['description', 'organism'])
.agg(num_misses=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['num_misses'], ascending=False)
.rename(columns={'description' : 'Culture Type',
                 'organism' : 'Infection Type',
                 'num_misses' : 'Number of Misses'})
)
df_frequent_misses_alg_high_level.to_csv('alg_misses_by_infection_type.csv', index=None)
df_frequent_misses_alg_high_level

### Dr misses stratified by what antibiotic was delivered

In [ ]:
df_frequent_misses_dr = (df_ast_labels_by_bug
.query("dr_covered == 0", engine='python')
.assign(description = lambda x: [format_description(a) for a in x.description])
.assign(organism = lambda x: [a.capitalize() for a in x.organism])
.assign(organism=lambda x: [map_high_level_organism[a] for a in x.organism])
.groupby(['description', 'organism', 'med_description'])
.agg(num_misses=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['num_misses'], ascending=False)
.rename(columns={'description' : 'Culture Type',
                 'organism' : 'Infection Type',
                 'med_description' : 'Antibiotics Administered',
                 'num_misses' : 'Number of Misses'})
)
df_frequent_misses_dr.to_csv('doctor_misses_by_infection_type_and_abx.csv', index=None)
df_frequent_misses_dr

### Alg misses by what antibiotic was given

In [ ]:
df_frequent_misses_alg = (df_ast_labels_by_bug
.query("alg_covered == 0", engine='python')
.assign(description = lambda x: [format_description(a) for a in x.description])
.assign(organism = lambda x: [a.capitalize() for a in x.organism])
.assign(organism=lambda x: [map_high_level_organism[a] for a in x.organism])
.groupby(['description', 'organism', 'IP_med_description'])
.agg(num_misses=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['num_misses'], ascending=False)
.rename(columns={'description' : 'Culture Type',
                 'organism' : 'Infection Type',
                 'IP_med_description' : 'Antibiotics Administered',
                 'num_misses' : 'Number of Misses'})
)
df_frequent_misses_alg.to_csv('alg_misses_by_infection_type_and_abx.csv', index=None)
df_frequent_misses_alg

## IV AND IM Antibiotics Only

In [ ]:
query = """
SELECT
    om.anon_id, om.pat_enc_csn_id_coded, om.order_med_id_coded, l.index_time, om.med_description,
    l.Ampicillin, l.Ciprofloxacin, l.Cefazolin, l.Ceftriaxone, l.Cefepime, l.Zosyn, l.Vancomycin,
    l.Meropenem, l.Vancomycin_Meropenem, l.Vancomycin_Zosyn, l.Vancomycin_Cefepime, l.Vancomycin_Ceftriaxone
FROM
    `mining-clinical-decisions.abx.abx_orders_given_and_stopped` om
INNER JOIN 
    `mining-clinical-decisions.abx.final_ast_labels` l
USING
    (pat_enc_csn_id_coded)
WHERE
    om.was_given = 1
ORDER BY 
    om.anon_id, om.pat_enc_csn_id_coded, om.order_time
"""
query_job = client.query(query)
df_abx = query_job.result().to_dataframe()
df_abx.head()

# Lambda that aggregate Antibiotic orders after we've grouped by CSN
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))

# Aggregate abx orders
df_drugs = (df_abx
    .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description]) # Only Take first word (abx)
    .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                        .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                    for a in x.med_description])
    .assign(year=lambda x: x.index_time.dt.year) # get year of each CSN - used to filter later on
    .groupby('pat_enc_csn_id_coded')
    .agg({'med_description' : concat_abx,
        'year' : 'first',
        'Ampicillin' : 'first',
        'Ciprofloxacin' : 'first',
        'Cefazolin' : 'first',
        'Ceftriaxone' : 'first',
        'Cefepime' : 'first',
        'Zosyn' : 'first',
        'Vancomycin' : 'first',
        'Meropenem' : 'first',
        'Vancomycin_Ceftriaxone' : 'first',
        'Vancomycin_Cefepime' : 'first',
        'Vancomycin_Zosyn' : 'first',
        'Vancomycin_Meropenem' : 'first'})
    .reset_index()
    # Only look at test set data and CSNs where allowed antibiotic selection was administered
    .query("year == 2019", engine='python') # and med_description in @abx_map_inverse", engine='python') 
    #.assign(med_description=lambda x: [abx_map_inverse[a] for a in x.med_description])
)

In [ ]:
df_drugs.head()

In [ ]:
len(df_drugs)

In [ ]:
unique_regimens = (df_drugs
.groupby('med_description')
.agg(num_ed_visits=('pat_enc_csn_id_coded', 'nunique'))
.sort_values('num_ed_visits', ascending=False)
.reset_index()
)
unique_regimens.head(20)

In [ ]:
unique_regimens.to_csv('unique_regimens.csv', index=None)

## Now Include Oral Abx that were listed as administered

In [ ]:
query = """
SELECT
    om.anon_id, om.pat_enc_csn_id_coded, om.order_med_id_coded, l.index_time, om.med_description,
    l.Ampicillin, l.Ciprofloxacin, l.Cefazolin, l.Ceftriaxone, l.Cefepime, l.Zosyn, l.Vancomycin,
    l.Meropenem, l.Vancomycin_Meropenem, l.Vancomycin_Zosyn, l.Vancomycin_Cefepime, l.Vancomycin_Ceftriaxone
FROM
    `mining-clinical-decisions.abx.abx_orders_given_and_stopped_include_oral` om
INNER JOIN 
    `mining-clinical-decisions.abx.final_ast_labels` l
USING
    (pat_enc_csn_id_coded)
WHERE
    om.was_given = 1
ORDER BY 
    om.anon_id, om.pat_enc_csn_id_coded, om.order_time
"""
query_job = client.query(query)
df_abx = query_job.result().to_dataframe()
df_abx.head()

# Lambda that aggregate Antibiotic orders after we've grouped by CSN
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))

# Aggregate abx orders
df_drugs = (df_abx
    .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description]) # Only Take first word (abx)
    .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                        .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                    for a in x.med_description])
    .assign(year=lambda x: x.index_time.dt.year) # get year of each CSN - used to filter later on
    .groupby('pat_enc_csn_id_coded')
    .agg({'med_description' : concat_abx,
        'year' : 'first',
        'Ampicillin' : 'first',
        'Ciprofloxacin' : 'first',
        'Cefazolin' : 'first',
        'Ceftriaxone' : 'first',
        'Cefepime' : 'first',
        'Zosyn' : 'first',
        'Vancomycin' : 'first',
        'Meropenem' : 'first',
        'Vancomycin_Ceftriaxone' : 'first',
        'Vancomycin_Cefepime' : 'first',
        'Vancomycin_Zosyn' : 'first',
        'Vancomycin_Meropenem' : 'first'})
    .reset_index()
    # Only look at test set data and CSNs where allowed antibiotic selection was administered
    .query("year == 2019", engine='python') # and med_description in @abx_map_inverse", engine='python') 
    #.assign(med_description=lambda x: [abx_map_inverse[a] for a in x.med_description])
)

In [ ]:
df_drugs.head()

In [ ]:
len(df_drugs)

In [ ]:
unique_regimens = (df_drugs
.groupby('med_description')
.agg(num_ed_visits=('pat_enc_csn_id_coded', 'nunique'))
.sort_values('num_ed_visits', ascending=False)
.reset_index()
)
unique_regimens.head(20)

In [ ]:
len(unique_regimens)

In [ ]:
unique_regimens.to_csv('unique_regimens_include_oral.csv', index=None)

### Inspect Misses

In [ ]:
query="""
SELECT DISTINCT
  a.anon_id, a.pat_enc_csn_id_coded, a.index_time, cs.organism, cs.description
FROM  
  `mining-clinical-decisions.abx.final_ast_labels` a
INNER JOIN
  `mining-clinical-decisions.abx.culture_orders_within_24_hrs` cult_orders
USING
  (pat_enc_csn_id_coded)
INNER JOIN 
  `som-nero-phi-jonc101.shc_core.culture_sensitivity` cs
USING
  (order_proc_id_coded)
WHERE 
    EXTRACT(YEAR FROM a.index_time) = 2019
AND 
    organism NOT LIKE "%COAG NEGATIVE STAPHYLOCOCCUS%"
AND 
    organism NOT LIKE "%CANDIDA%"


""" 
query_job = client.query(query)
df_bugs = query_job.result().to_dataframe()

In [ ]:
df_bugs.head()

In [ ]:
(df_bugs
.groupby('description')
.agg(num_ids=('pat_enc_csn_id_coded', 'nunique'))
)

### Bugs that are most frequent misses

In [ ]:
import pdb
### Get organisms corresponding to misses
concat_bugs = lambda x : ' '.join(np.unique(sorted([a.capitalize() for a in x])))
def group_cultures(x):
    if x.split()[0] in ("BLOOD", "URINE"): 
        return x.split()[0].capitalize()
    else:
        return 'Other fluid' 
        
def pipe_func(df):
    df = df.assign(organism_and_culture = df.apply(lambda x: x['description'] + ' ' + x['organism'], axis=1))
    return df

df_bugs_collapsed = (df_bugs
.assign(description = lambda x: [group_cultures(a) for a in x.description]) 
.pipe(pipe_func)
.groupby('pat_enc_csn_id_coded')
.agg(organism=('organism_and_culture', concat_bugs))
.reset_index()
)


df_bugs_collapsed = (df_bugs_collapsed
.merge(df_drugs, how='inner', on='pat_enc_csn_id_coded')
.query("dr_covered == 0", engine='python')
.groupby('organism')
.agg(num_csns=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values('num_csns', ascending=False)
)
df_bugs_collapsed

### Bug & Drug Combos of misses

In [ ]:
### Get organisms corresponding to misses
df_bugs_collapsed = (df_bugs
.assign(description = lambda x: [group_cultures(a) for a in x.description]) 
.pipe(pipe_func)
.groupby('pat_enc_csn_id_coded')
.agg(organism=('organism_and_culture', concat_bugs))
.reset_index()
)

df_misses_with_drugs = (df_bugs_collapsed
.merge(df_drugs, how='inner', on='pat_enc_csn_id_coded')
.query("dr_covered == 0", engine='python')
.groupby(['organism', 'med_description'])
.agg(num_csns=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values('num_csns', ascending=False)
)

df_misses_with_drugs

In [ ]:
df_misses_with_drugs.to_csv("top_misses_with_what_was_given.csv", index=None)